

   "![FMD_Overview](https://github.com/edkreuk/FMD_FRAMEWORK/blob/main/Images/FMD_Overview.png?raw=true)"

Create Lakehouse if not exists and Attach Lakehouse 

In [317]:
try:
    notebookutils.lakehouse.create(name = "LH_FMD_CONFIGURATION")
except Exception as ex:
    print('Lakehouse already exists')

StatementMeta(, 3e96f9db-6801-4dbb-8609-8c3d1bc7a8c1, 217, Finished, Available, Finished)

Lakehouse already exists


In [ ]:
%%configure -f
{ 
    "defaultLakehouse": { 
        "name":  "LH_FMD_ONFIGURATION"
    }
}

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, -1, Finished, Available, Finished)

In [414]:
deploy_from_github = False
driver = '{ODBC Driver 18 for SQL Server}'
# target connections guid 
# add the correct id for every connections
# change the capacity id
# change the workspace roles

FrameworkName= 'DEMO1' # max 6 characters for better visibility, no spaces and the end of the name
#'560C908C-072A-478C-939C-CFC4398D2FDB
capacity_id_dvlm = '560C908C-072A-478C-939C-CFC4398D2FDB'       # Which capacity will be used for these workspaces in development
capacity_id_prod = '560C908C-072A-478C-939C-CFC4398D2FDB'       # Which capacity will be used for these workspaces in production
capacity_id_config = '2BDC6A16-1EE7-4BAD-9127-5EF6BEB00E84'     #'Which capacity will be used for this workspace for the FMD Database
capacity_id_log = '560C908C-072A-478C-939C-CFC4398D2FDB'        #'Which capacity will be used for this workspace for the LOG Warehouse

workspace_roles = [ # Keep emtpy [] if you only want to assign this to your personal account
                    {
                        "principal": {
                            "id": "ba4a392b-9596-46d8-9486-53eb2c2b22bf",
                            "displayName": "sg-fabric-contributor",
                            "type": "Group"
                        },
                        "role": "Contributor"  # Admin, Member, Contributor, Viewer
                        },
                        {
                        "principal": {
                            "id": "39b98289-1540-4866-b9f0-3935f9eeebd6",
                            "displayName": "sg-fabric-admin",
                            "type": "Group"
                        },
                        "role": "Admin"
                        }
                    ]

logging = {
                    'workspace': {
                        'name' : FrameworkName + ' LOG FMD', # Name of target workspace
                        'roles' : workspace_roles,
                        'capacity_id' : capacity_id_log
                    },
                    'WarehouseName' : 'WH_LOG' # Name of target logging warehouse
}

configuration = {
                    'workspace': {
                        'name' : FrameworkName + ' CONFIG FMD', # Name of target workspace
                        'roles' : workspace_roles,
                        'capacity_id' : capacity_id_config
                    },
                    'DatabaseName' : 'SQL_FMD_FRAMEWORK' # Name of target configuration SQL Database
}

environments = [
                    {
                        'environment_name' : 'development', # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : FrameworkName + ' DATA (D) FMD', # Name of target code workspace for development
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_dvlm
                            },
                            'code' : {
                                'name' : FrameworkName + ' CODE (D) FMD', # Name of target data workspace for development
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_dvlm
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e', # Required Guid to the Fabric SQL connection
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7', # Required Guid to the Fabric datapipelines connection
                            'CON_FMD_ASQL_01' : 'cf673e6a-13f6-4ebb-9cbb-4ba4ab390818', # Optional Guid to an Azure SQL database connection
                            'CON_FMD_ASQL_02' : '11a8e5fe-fbca-4822-9ba4-9162cf56e6dd', # Optional Guid to an second Azure SQL database connection
                            'CON_FMD_ADLS_01' : 'a0581b6e-5e38-46eb-bab2-7f08e9a35c30', # Optional Guid to an Azure SQL Datalake storage connection
                            'CON_FMD_ADF_PIPELINES' : 'e93f565a-e2bc-4b60-900e-1907e825e37c' # Optional Guid to an Azure Datafactory connection
                        }
                    },
                    {
                        'environment_name' : 'production', # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : FrameworkName + ' DATA (P) FMD',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_prod
                            },
                            'code' : {
                                'name' : FrameworkName + ' CODE (P) FMD',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id_prod
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',
                            'CON_FMD_ASQL_01' : 'cf673e6a-13f6-4ebb-9cbb-4ba4ab390818',
                            'CON_FMD_ASQL_02' : '11a8e5fe-fbca-4822-9ba4-9162cf56e6dd',
                            'CON_FMD_ADLS_01' : 'a0581b6e-5e38-46eb-bab2-7f08e9a35c30',
                            'CON_FMD_ADF_PIPELINES' : 'e93f565a-e2bc-4b60-900e-1907e825e37c'
                        }
                    }
                ]



StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 110, Finished, Available, Finished)

## Import Libraries

In [415]:
from json import loads, dumps
import json
import requests
import base64
import time
import uuid
import struct
import pyodbc

from typing import Callable, List, Dict, Optional, Any
from datetime import datetime
from time import sleep, time
from dataclasses import dataclass, field

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 111, Finished, Available, Finished)

## Download Configuration File from Github or upload the file manually 

**Manually**: upload to Files/deployment/FMD_deployment.json

In [416]:
# Open deployment json file
deployment_manifest = {}
if deploy_from_github:
    print("Downloading from Github to FMD_FRAMEWORK")
    url = 'https://raw.githubusercontent.com/edkreuk/FMD_FRAMEWORK/main/FMD_deployment.json'
    github_download = requests.get(url)
    folder_path = notebookutils.fs.getMountPath('/default') + "/Files/deployment/"
    notebookutils.fs.mkdirs(f"file://" +folder_path)
    with open(folder_path + "FMD_deployment.json", "w") as f:
        f.write(json.dumps(github_download.json()))
    

    print("Read from FMD_FRAMEWORK Github")


# Read deployment manifest

with open(f"{notebookutils.fs.getMountPath('/default')}/Files/deployment/FMD_deployment.json") as f:
    deployment_manifest = json.load(f)
    print("Read from FMD_FRAMEWORK LOCAL")


StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 112, Finished, Available, Finished)

Read from FMD_FRAMEWORK LOCAL


## Load deployment functions

In [417]:
%run NB_FMD_DEPLOYMENT_UTILS

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 116, Finished, Available, Finished)

In [418]:
tasks=[]

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 117, Finished, Available, Finished)

In [419]:
fmd_api_access_token =  notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api')
fabric_session = create_fabric_session(fabric_token = fmd_api_access_token)

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 118, Finished, Available, Finished)

In [420]:
# check if token is valid
for token in [fmd_api_access_token]:
    if not token:
        continue
    header, payload, signature = token.split('.')
    payload += '=' * (-len(payload) % 4)  # Add padding
    token_dict = loads(base64.urlsafe_b64decode(payload))
    directory_id = token_dict.get("tid")
    timest = token_dict.get("exp")
    expiry = (datetime.fromtimestamp(timest) - datetime.now()).total_seconds() // 60
    expiry_str = str(expiry) if expiry < 5 else str(expiry)
    print(F"token {directory_id} will expire in {expiry_str} minutes at\t{datetime.fromtimestamp(timest)} UTC")
print(F"Current time:\t\t\t\t\t\t\t\t\t{datetime.now().replace(microsecond=0)} UTC")

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 119, Finished, Available, Finished)

token c183ff6a-8ca1-4185-9378-af980978377c will expire in 24.0 minutes at	2025-03-12 22:26:43 UTC
Current time:									2025-03-12 22:01:59 UTC


## Start Deployment

In [421]:
# Create necessary workspaces 
start = time()

for environment in environments:

    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    
    # Loop through the workspace names and get their IDs
    
    for workspace in [environment['workspaces']['data'], environment['workspaces']['code'], configuration['workspace'], logging['workspace']]:
        
        print(f" -----")
        print(f" - Processing: data workspace {environment['environment_name']}")
        
        # List all workspaces
        workspaces_current = get_fabric_workspaces(fabric_session)        
        
        # Check if the displayName exists in the workspaces
        matching_workspaces = [workspace_current for workspace_current in workspaces_current.get('value') if workspace_current['displayName'] == workspace['name']]
        
        if matching_workspaces:
            print(f" - Workspace '{workspace['name']}' found. Workspace ID: {matching_workspaces[0]['id']}")
            workspace['id'] = matching_workspaces[0]['id']
        else:
            print(f" - Workspace '{workspace['name']}' not found. Creating new workspace...")
            workspace_created = fabric_request(fabric_session, F"workspaces/", 'POST', payload={"displayName": workspace['name']}, payloadtype='json')
            workspace['id'] = workspace_created['id']
            tasks.append({"task_name":f"create item {workspace['name']} initially", "task_duration": int(time() - start), "status": "success"})
        
        assign_fabric_workspace_capacity(fabric_session, workspace['id'], workspace['capacity_id'])
        tasks.append({"task_name":f"Workspace '{workspace['name']}' connected to capacity ID: {workspace['capacity_id']}", "task_duration": int(time() - start), "status": "success"})
        print(f" - Workspace '{workspace['name']}' created with ID: {workspace['id']} and capacity_id: {workspace['capacity_id']}")

        # Check if roles exists or create them
        print(f" - Assiging Workspace roles")
        assign_fabric_workspace_roles(fabric_session, workspace['id'], workspace['roles'])

    # Print the workspace IDs
    print(f"--------------------------")
    print(f"Workspace ID for data workspace {environment['environment_name']}: {environment['workspaces']['data']['id']}")
    print(f"Workspace ID for code workspace {environment['environment_name']}: {environment['workspaces']['code']['id']}")
    print(f"Workspace ID for config workspace: {configuration['workspace']['id']}")
    print(f"Workspace ID for logging workspace: {logging['workspace']['id']}")


StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 120, Finished, Available, Finished)

--------------------------
Processing: development
 -----
 - Processing: data workspace development
 - Workspace 'DEMO1 DATA (D) FMD' found. Workspace ID: 556754dc-f7eb-4ddf-9bf8-a2c4fa252c94
Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (556754dc-f7eb-4ddf-9bf8-a2c4fa252c94)
Request accepted: Dont Wait for result
 - Workspace 'DEMO1 DATA (D) FMD' created with ID: 556754dc-f7eb-4ddf-9bf8-a2c4fa252c94 and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 556754dc-f7eb-4ddf-9bf8-a2c4fa252c94
Skipping: sg-fabric-admin for 556754dc-f7eb-4ddf-9bf8-a2c4fa252c94
 -----
 - Processing: data workspace development
 - Workspace 'DEMO1 CODE (D) FMD' found. Workspace ID: 208c0835-9575-4b05-b170-5640973088e7
Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (208c0835-9575-4b05-b170-5640

In [422]:
# re-map databases

for target_item in deployment_manifest['logging']['items']:
    if target_item['type'] in ('Warehouse', 'SQLEndpoint'):
        target_item['displayName'] = logging['WarehouseName']

for target_item in deployment_manifest['configuration']['items']:
    if target_item['type'] in ('SQLDatabase','SQLEndpoint'):
        target_item['displayName'] = configuration['DatabaseName']


StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 121, Finished, Available, Finished)

In [423]:
def workspace_deployment(workspace_deployment_config, workspace_deployment_items, replace_collection, empty = True):
    print(f" --------------------------")
    print(f" Processing code workspace {workspace_deployment_config['name']}")
    
    target_items = get_fabric_items(fabric_session, workspace_deployment_config['id'])

    for deployment_item in workspace_deployment_items:
        for target_item in target_items['value']:
            if target_item['displayName'] == deployment_item['displayName'] \
                    and target_item['type'] == deployment_item['type']:
                print(f" - Skip existing: {deployment_item['displayName']}, {deployment_item['type']}, {target_item['id']}")
                break
        else:
            #if deployment_item['type'] == 'SQLDatabase':
            #    raise Exception(f"Please create manually the SQL database {deployment_item['displayName']} in the configuration workspace {configuration['workspace']['name']} and re-run this code.")  
                # skip because sqldatabase isn't supported yet
            
            print(f" - Creating: {deployment_item['displayName']} {deployment_item['type']}")
            item = deployment_item.copy()
            
            if empty:
                if item.get('definition'):
                    print(f" - Dropping definition")
                    item.pop('definition')

            target_item = fabric_request(fabric_session, url=f"workspaces/{workspace_deployment_config['id']}/items/", method="POST", payload=item, payloadtype='json')

        deployment_item["new_id"] = target_item['id']

        if deployment_item['type'] in ('Warehouse', 'SQLDatabase'):
            if deployment_item.get('org_endpoint', '') != '':
                return_item = fabric_request(fabric_session, url=f"workspaces/{workspace_deployment_config['id']}/{deployment_item['type']}s/{target_item['id']}", method="GET")
                if deployment_item['type'] in ('Warehouse'):
                    if return_item.get("properties", {}).get("connectionString", "") != '':
                        deployment_item["connectionString"] = return_item["properties"]["connectionString"]
                        replace_collection.append({"old_id": deployment_item["org_endpoint"], "new_id": deployment_item["connectionString"]})
                
                if deployment_item['type'] in ('SQLDatabase'):
                    if return_item.get("properties", {}).get("serverFqdn", "") != '':
                        deployment_item["connectionString"] = return_item["properties"]["serverFqdn"].replace(',1433', '')
                        replace_collection.append({"old_id": deployment_item["org_endpoint"], "new_id": deployment_item["connectionString"]})
                    if return_item.get("properties", {}).get("databaseName", "") != '':
                        deployment_item["databaseName"] = return_item["properties"]["databaseName"]

        replace_collection.append({"old_id": deployment_item["org_id"], "new_id": deployment_item["new_id"]})

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 122, Finished, Available, Finished)

In [424]:
# create empty items if not exists
    
for environment in environments:

    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    environment['guids_to_replace'] = []
    workspace_deployment(environment['workspaces']['code'], deployment_manifest['items'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_code"], "new_id": environment['workspaces']['code']['id']})
    workspace_deployment(environment['workspaces']['data'], deployment_manifest['data'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_data"], "new_id": environment['workspaces']['data']['id']})



StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 123, Finished, Available, Finished)

--------------------------
Processing: development
 --------------------------
 Processing code workspace DEMO1 CODE (D) FMD
 - Skip existing: ENV_FMD, Environment, de1751e0-043c-4a2e-9fdc-6e62df0003d8
 - Skip existing: NB_FMD_LOAD_BRONZE_SILVER, Notebook, 5f4f6dd8-cf3a-4e9b-bb38-b75f38e75a8c
 - Skip existing: NB_FMD_LOAD_LANDING_BRONZE, Notebook, 2c4155da-aab8-45ee-b2ca-c3927f2d63d6
 - Skip existing: PL_FMD_LOAD_LANDINGZONE, DataPipeline, e2249af1-0605-4db6-835c-5fc304d515f7
 - Skip existing: PL_FMD_LOAD_ALL, DataPipeline, 61d79c1f-5ca6-419d-94de-1b88519e1aee
 - Skip existing: PL_FMD_LOAD_BRONZE, DataPipeline, c58ef9f9-5ef6-4644-a274-d9b39bb678bc
 - Skip existing: PL_FMD_LOAD_SILVER, DataPipeline, 7cbe6476-c470-42bd-b34a-191745fb21d4
 - Skip existing: PL_FMD_LDZ_COPY_FROM_ASQL_01, DataPipeline, bab73525-86ad-40a8-be73-762c38e8f618
 - Skip existing: PL_FMD_LDZ_COPY_FROM_ONELAKE_TABLES_01, DataPipeline, 712c38ca-e1c9-4c14-a1ba-844459697490
 - Skip existing: PL_FMD_LDZ_COPY_FROM_ADLS_01,

In [425]:
def create_fabric_database():
    try:
        workspace_deployment(configuration['workspace'], deployment_manifest['configuration']['items'], environment['guids_to_replace'], True)
        environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_config"], "new_id": configuration['workspace']['id']})
    except Exception as e:
        print(f"Error: {e}. Retrying in 60 seconds...")
        sleep(60)
        create_fabric_database()

create_fabric_database()




StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 124, Finished, Available, Finished)

 --------------------------
 Processing code workspace DEMO1 CONFIG FMD
 - Skip existing: SQL_FMD_FRAMEWORK, SQLDatabase, ab948fad-1fdf-444c-9625-ff76a45ac222
 - Skip existing: SQL_FMD_FRAMEWORK, SQLEndpoint, ce2b9a35-35b4-42b1-8e05-d9b234e4e492


In [426]:
def create_fabric_warehouse():
    try:
        workspace_deployment(logging['workspace'], deployment_manifest['logging']['items'], environment['guids_to_replace'], True)
        environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_logging"], "new_id": logging['workspace']['id']})
    except Exception as e:
        print(f"Error: {e}. Retrying in 60 seconds...")
        time.sleep(60)
        create_fabric_warehouse()

create_fabric_warehouse()

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 125, Finished, Available, Finished)

 --------------------------
 Processing code workspace DEMO1 LOG FMD
 - Skip existing: WH_LOG, Warehouse, 0b10abba-ba1e-4eb7-b9e6-124fdca49562


In [427]:
#print(environment['guids_to_replace'])

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 126, Finished, Available, Finished)

In [428]:
items_to_deploy = deployment_manifest["items"]

for environment in environments:
    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    
    # Append the remaining pairs
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_SQL"], "new_id": environment['connections']['CON_FMD_FABRIC_SQL']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_PIPELINES"], "new_id": environment['connections']['CON_FMD_FABRIC_PIPELINES']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ASQL_01"], "new_id": environment['connections']['CON_FMD_ASQL_01']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ASQL_02"], "new_id": environment['connections']['CON_FMD_ASQL_02']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADLS_01"], "new_id": environment['connections']['CON_FMD_ADLS_01']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADF_PIPELINES"], "new_id": environment['connections']['CON_FMD_ADF_PIPELINES']})
    
    # Deploy items to workspace
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    deploy_items(items_to_deploy, environment['guids_to_replace'], fmd_api_access_token, environment['workspaces']['code']['id'], existing_items["value"])

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 127, Finished, Available, Finished)

--------------------------
Processing: development
Deploy ENV_FMD (Environment)
Update de1751e0-043c-4a2e-9fdc-6e62df0003d8
Deploy NB_FMD_LOAD_BRONZE_SILVER (Notebook)
Update 5f4f6dd8-cf3a-4e9b-bb38-b75f38e75a8c
Deploy NB_FMD_LOAD_LANDING_BRONZE (Notebook)
Update 2c4155da-aab8-45ee-b2ca-c3927f2d63d6
Deploy PL_FMD_LOAD_LANDINGZONE (DataPipeline)
Update e2249af1-0605-4db6-835c-5fc304d515f7
Deploy PL_FMD_LOAD_ALL (DataPipeline)
Update 61d79c1f-5ca6-419d-94de-1b88519e1aee
Deploy PL_FMD_LOAD_BRONZE (DataPipeline)
Update c58ef9f9-5ef6-4644-a274-d9b39bb678bc
Deploy PL_FMD_LOAD_SILVER (DataPipeline)
Update 7cbe6476-c470-42bd-b34a-191745fb21d4
Deploy PL_FMD_LDZ_COPY_FROM_ASQL_01 (DataPipeline)
Update bab73525-86ad-40a8-be73-762c38e8f618
Deploy PL_FMD_LDZ_COPY_FROM_ONELAKE_TABLES_01 (DataPipeline)
Update 712c38ca-e1c9-4c14-a1ba-844459697490
Deploy PL_FMD_LDZ_COPY_FROM_ADLS_01 (DataPipeline)
Update b449d9b6-295a-4552-9542-a67ebd1859f5
Deploy PL_FMD_LDZ_COPY_FROM_ASQL_02 (DataPipeline)
Update 9f33

In [429]:
# by timeout re-run

for target_item in deployment_manifest['logging']['items']:
    if target_item['type'] == 'Warehouse':
        connstring = target_item["connectionString"]
        database = target_item['displayName']

try:
    i = 0
    token = notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    print(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};")
    connection = pyodbc.connect(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};", attrs_before={1256:token_struct}, timeout=12)

    with connection.cursor() as cursor:
        cursor.execute("SELECT 1")  # Execute the warm-up query (a simple query like 'SELECT 1' can be used)
        cursor.fetchone()
        connection.timeout = 5  # Setting a lower timeout for subsequent queries
        for i, query in enumerate(deployment_manifest["logging"]["queries"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()

            tasks.append({"task_name": f"{target_item.get('displayName')} {database} query {i}", "task_duration": 1, "status": "success"})
except pyodbc.OperationalError as e:
    print(e)
    tasks.append({"task_name": f"{target_item.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"pyodbc failed: {e}"})
except Exception as e:
    print(e)
    tasks.append({"task_name": f"{target_item.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"failed: {e}"})

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 128, Finished, Available, Finished)

DRIVER={ODBC Driver 18 for SQL Server};SERVER=nl7yhqnbrscude3yv6mas6bxpq-fb5ew36gdamexgepvv6ne45j2a.datawarehouse.fabric.microsoft.com;PORT=1433;DATABASE=WH_LOG;
 - execute "
    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'logging')
    BEGIN
        EXEC('CREATE SCHEMA [logging];');
    END
    "
 - execute "
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'PipelineExecution' AND schema_id = SCHEMA_ID('logging'))
    BEGIN
        CREATE TABLE [logging].[PipelineExecution]
        (
            [WorkspaceGuid] [uniqueidentifier]  NULL,
            [PipelineRunGuid] [uniqueidentifier]  NULL,
            [PipelineParentRunGuid] [uniqueidentifier]  NULL,
            [PipelineGuid] [uniqueidentifier]  NULL,
            [PipelineName] [varchar](100)  NULL,
            [PipelineParameters] [varchar](8000)  NULL,
            [TriggerType] [varchar](50)  NULL,
            [TriggerGuid] [uniqueidentifier]  NULL,
            [TriggerTime] [datetime2](6)  NULL,
            [L

In [430]:
deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "00000000-0000-0000-0000-000000000000", @Name = "CON_FMD_ONELAKE", @Type = "ONELAKE", @IsActive = 1')
deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ASQL_01"])}", @Name = "CON_FMD_ASQL_01", @Type = "SQL", @IsActive = 1')
deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ASQL_02"])}", @Name = "CON_FMD_ASQL_02", @Type = "SQL", @IsActive = 1')
deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ADLS_01"])}", @Name = "CON_FMD_ADLS_01", @Type = "ADLS", @IsActive = 1')
deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ADF_PIPELINES"])}", @Name = "CON_FMD_ADF_PIPELINES", @Type = "ADF", @IsActive = 1')

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 129, Finished, Available, Finished)

In [431]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE')
    DECLARE @ConnectionIdInternal INT = (SELECT ConnectionId FROM integration.Connection WHERE ConnectionGuid = '00000000-0000-0000-0000-000000000000')
    EXECUTE [integration].[sp_UpsertDataSource] 
        @ConnectionId = @ConnectionIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@Name = 'LH_DATA_LANDINGZONE'
        ,@Namespace = 'ONELAKE'
        ,@Type = 'ONELAKE_TABLES_01'
        ,@Description = 'ONELAKE_TABLES'
        ,@IsActive = 1
""")

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 130, Finished, Available, Finished)

In [432]:
workspaces = []
workspaces.append(logging['workspace'])
workspaces.append(configuration['workspace'])

for environment in environments:
    workspaces.append(environment['workspaces']['code'])
    workspaces.append(environment['workspaces']['data'])
    
for workspace in workspaces:
    print(f'EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "{workspace["id"]}" ,@Name = "{workspace["name"]}")')
    deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertWorkspace] @WorkspaceId = "{workspace["id"]}", @Name = "{workspace["name"]}"')

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 131, Finished, Available, Finished)

EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "6f4b7a28-18c6-4b18-988f-ad7cd273a9d0" ,@Name = "DEMO1 LOG FMD")
EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "75c9bcc7-e074-4ca7-9a7e-76eacc2c94ea" ,@Name = "DEMO1 CONFIG FMD")
EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "208c0835-9575-4b05-b170-5640973088e7" ,@Name = "DEMO1 CODE (D) FMD")
EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "DEMO1 DATA (D) FMD")
EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "e71cb5df-b718-4cb3-8cb6-07abddb65f38" ,@Name = "DEMO1 CODE (P) FMD")
EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "254a1222-703f-46e2-adb3-ac40a67421ec" ,@Name = "DEMO1 DATA (P) FMD")


In [433]:
for environment in environments:
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    for item in existing_items.get('value', []):
        if item['type'] == 'DataPipeline':
            print(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 132, Finished, Available, Finished)

EXEC [integration].[sp_UpsertPipeline] @PipelineId = "e2249af1-0605-4db6-835c-5fc304d515f7", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "PL_FMD_LOAD_LANDINGZONE"
EXEC [integration].[sp_UpsertPipeline] @PipelineId = "61d79c1f-5ca6-419d-94de-1b88519e1aee", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "PL_FMD_LOAD_ALL"
EXEC [integration].[sp_UpsertPipeline] @PipelineId = "c58ef9f9-5ef6-4644-a274-d9b39bb678bc", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "PL_FMD_LOAD_BRONZE"
EXEC [integration].[sp_UpsertPipeline] @PipelineId = "7cbe6476-c470-42bd-b34a-191745fb21d4", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "PL_FMD_LOAD_SILVER"
EXEC [integration].[sp_UpsertPipeline] @PipelineId = "bab73525-86ad-40a8-be73-762c38e8f618", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "PL_FMD_LDZ_COPY_FROM_ASQL_01"
EXEC [integration].[sp_UpsertPipeline] @PipelineId = "712c38ca-e1c9-4c14-a1ba-844459697490", @Workspace

In [434]:
for environment in environments:
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['data']['id']}/items/", method="GET")
    for item in existing_items.get('value', []):
        if item['type'] == 'Lakehouse':
            print(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            deployment_manifest["configuration"]["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 133, Finished, Available, Finished)

EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "daf9b505-61a8-4fa0-ab16-ae3ee5db0bd8", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "LH_DATA_LANDINGZONE"
EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "367b1765-b0ac-4f08-9d6c-cf72522ed14f", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "LH_BRONZE_LAYER"
EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "972b0750-d1e5-4d59-b1c0-9bffdc6dd789", @WorkspaceId = "556754dc-f7eb-4ddf-9bf8-a2c4fa252c94" ,@Name = "LH_SILVER_LAYER"
EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "b06c6404-9533-44af-8571-f92c3925ac91", @WorkspaceId = "254a1222-703f-46e2-adb3-ac40a67421ec" ,@Name = "LH_DATA_LANDINGZONE"
EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "037d2f80-e858-4c5b-ae0e-e4419052876d", @WorkspaceId = "254a1222-703f-46e2-adb3-ac40a67421ec" ,@Name = "LH_BRONZE_LAYER"
EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "997f1ceb-d4e9-4b03-8138-349f69f9e647", @WorkspaceId = "2

In [435]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_DATA_LANDINGZONE')
    EXECUTE [integration].[sp_UpsertLandingzoneEntity] 
        @LandingzoneEntityId = @LandingzoneEntityIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@LakehouseId = @LakehouseIdInternal
        ,@SourceSchema = 'in'
        ,@SourceName = 'customer'
        ,@FileName = 'customer'
        ,@FilePath = 'in'
        ,@FileType = 'parquet'
        ,@IsIncremental = 0
        ,@IsIncrementalColumn = ''
        ,@IsActive = 1
""")

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 134, Finished, Available, Finished)

In [436]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
    DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_BRONZE_LAYER')
    EXECUTE [integration].[sp_UpsertBronzeLayerEntity] 
        @BronzeLayerEntityId = @BronzeLayerEntityIdInternal
        ,@LandingzoneEntityId = @LandingzoneEntityIdInternal
        ,@Schema = 'in'
        ,@Name = 'customer'
        ,@FileType = 'Delta'
        ,@LakehouseId = @LakehouseIdInternal
        ,@PrimaryKeys = 'CustomerId'
        ,@IsActive = 1
""")

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 135, Finished, Available, Finished)

In [437]:
deployment_manifest["configuration"]["queries_stored_procedures"].append("""
    DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @SilverLayerEntityIdInternal INT = (SELECT SilverLayerEntityId FROM integration.SilverLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_SILVER_LAYER')
    EXECUTE [integration].[sp_UpsertSilverLayerEntity] 
        @SilverLayerEntityId = @SilverLayerEntityIdInternal
        ,@BronzeLayerEntityId = @BronzeLayerEntityIdInternal
        ,@LakehouseId = @LakehouseIdInternal
        ,@Name = 'customer'
        ,@Schema = 'in'
        ,@FileType = 'delta'
        ,@IsActive = 1
""")

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 136, Finished, Available, Finished)

In [438]:
for target_item in deployment_manifest['configuration']['items']:
    if target_item['type'] == 'SQLDatabase':
        connstring = target_item["connectionString"]
        database = target_item['databaseName']

try:
    i = 0
    token = mssparkutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    print(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};")
    connection = pyodbc.connect(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};", attrs_before={1256:token_struct}, timeout=12)

    with connection.cursor() as cursor:
        cursor.execute("SELECT 1")  # Execute the warm-up query (a simple query like 'SELECT 1' can be used)
        cursor.fetchone()
        connection.timeout = 5  # Setting a lower timeout for subsequent queries
        for i, query in enumerate(deployment_manifest["configuration"]["queries_tables"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(deployment_manifest["configuration"]["queries_views"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(deployment_manifest["configuration"]["queries_stored_procedures"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"success"})
except pyodbc.OperationalError as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"pyodbc failed: {e}"})
except Exception as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"failed: {e}"})

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 137, Finished, Available, Finished)

DRIVER={ODBC Driver 18 for SQL Server};SERVER=nl7yhqnbrscude3yv6mas6bxpq-y66ms5lu4ctuzgt6o3vmyleu5i.database.fabric.microsoft.com;PORT=1433;DATABASE=SQL_FMD_FRAMEWORK-ab948fad-1fdf-444c-9625-ff76a45ac222;
 - execute "
    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'integration')
    BEGIN
        EXEC('CREATE SCHEMA [integration];');
    END
    "
 - execute "
    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'execution')
    BEGIN
        EXEC('CREATE SCHEMA [execution];');
    END
    "
 - execute "
    DECLARE @SQL NVARCHAR(MAX) = '';

    SELECT @SQL = @SQL + 'ALTER TABLE ' + QUOTENAME(s.name) + '.' + QUOTENAME(t.name) + 
    ' DROP CONSTRAINT ' + QUOTENAME(dc.name) + ';' + CHAR(13)
    FROM sys.default_constraints dc
    INNER JOIN sys.tables t ON dc.parent_object_id = t.object_id
    INNER JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE s.name IN ('integration', 'execution');

    EXEC sp_executesql @SQL
    "
 - execute "
    IF NOT EXISTS (SELECT * 

In [439]:
display(tasks)

StatementMeta(, 95e50a05-94ce-4a80-b142-46099c8476e7, 138, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c03c4840-c5f4-4e5f-a368-6e14c0aaea51)